In [364]:
import os
import numpy as np
import time
from models import ModelParams, ModelShocks, ModelSol
from equations import *
from equations_matrix import calc_X
from solvers import solve_price_and_cost
from functions import generate_rand_params
from optimization import reconstruct_w_hat, objective_w_hat_reduced
from scipy.optimize import minimize
from concurrent.futures import ProcessPoolExecutor, as_completed

# 1. Read the data of the ModelParams

In [365]:
data = np.load("real_data_2017.npz")
N, J = data["N"], data["J"]
mp = ModelParams(
    N=N,
    J=J,
    alpha=data["alpha"],
    beta=data["beta"],
    gamma=data["gamma"],
    theta=data["theta"],
    pif=data["pi_f"],
    pim=data["pi_m"],
    tilde_tau=data["tilde_tau"],
    Xf=np.ones((N, J)),
    Xm=np.ones((N, J)),
    w0=data["VA"],
    L0=np.ones_like(data["VA"]),
    td=data["D"],
)

# 2. Reconstruct the ModelShocks and ModelSol from the result

In [356]:
out_dir = "output/counterfactual"

In [373]:
country_dm_1_shocks = []
country_dm_1_sols = []
country_dm_2_shocks = []
country_dm_2_sols = []
for i in range(9):
    country_dm_1_shocks.append(ModelShocks.load_from_npz(os.path.join(out_dir, f"country_dm_1", f"result_{i}_shock.npz"), mp))
    country_dm_1_sols.append(ModelSol.load_from_npz(os.path.join(out_dir, f"country_dm_1", f"result_{i}_sol.npz"), mp, country_dm_1_shocks[i]))
    country_dm_2_shocks.append(ModelShocks.load_from_npz(os.path.join(out_dir, f"country_dm_2", f"result_{i}_shock.npz"), mp))
    country_dm_2_sols.append(ModelSol.load_from_npz(os.path.join(out_dir, f"country_dm_2", f"result_{i}_sol.npz"), mp, country_dm_1_shocks[i]))

In [374]:
sector_dm_1_shocks = []
sector_dm_1_sols = []
sector_dm_2_shocks = []
sector_dm_2_sols = []
for i in range(9):
    sector_dm_1_shocks.append(ModelShocks.load_from_npz(os.path.join(out_dir, f"sector_dm_1", f"result_{i}_shock.npz"), mp))
    sector_dm_1_sols.append(ModelSol.load_from_npz(os.path.join(out_dir, f"sector_dm_1", f"result_{i}_sol.npz"), mp, sector_dm_1_shocks[i]))
    sector_dm_2_shocks.append(ModelShocks.load_from_npz(os.path.join(out_dir, f"sector_dm_2", f"result_{i}_shock.npz"), mp))
    sector_dm_2_sols.append(ModelSol.load_from_npz(os.path.join(out_dir, f"sector_dm_2", f"result_{i}_sol.npz"), mp, sector_dm_1_shocks[i]))

In [375]:
idiosyncratic_dm_1_shocks = []
idiosyncratic_dm_1_sols = []
idiosyncratic_dm_2_shocks = []
idiosyncratic_dm_2_sols = []
for i in range(9):
    idiosyncratic_dm_1_shocks.append(ModelShocks.load_from_npz(os.path.join(out_dir, f"idiosyncratic_dm_1", f"result_{i}_shock.npz"), mp))
    idiosyncratic_dm_1_sols.append(ModelSol.load_from_npz(os.path.join(out_dir, f"idiosyncratic_dm_1", f"result_{i}_sol.npz"), mp, idiosyncratic_dm_1_shocks[i]))
    idiosyncratic_dm_2_shocks.append(ModelShocks.load_from_npz(os.path.join(out_dir, f"idiosyncratic_dm_2", f"result_{i}_shock.npz"), mp))
    idiosyncratic_dm_2_sols.append(ModelSol.load_from_npz(os.path.join(out_dir, f"idiosyncratic_dm_2", f"result_{i}_sol.npz"), mp, idiosyncratic_dm_1_shocks[i]))

# 3. Calculate the treatment effects under different types of shocks

In [377]:
country_effects = np.zeros((10, N))
sector_effects = np.zeros((10, N))
idiosyncratic_effects = np.zeros((10, N))

for i in range(9):
    country_effects[i, :] = calc_W(country_dm_2_sols[i]) / calc_W(country_dm_1_sols[i])
    sector_effects[i, :] = calc_W(sector_dm_2_sols[i]) / calc_W(sector_dm_1_sols[i])
    idiosyncratic_effects[i, :] = calc_W(idiosyncratic_dm_2_sols[i]) / calc_W(idiosyncratic_dm_1_sols[i])


In [388]:
mean_country_effects = np.mean(country_effects, axis=0)
std_country_effects = np.std(country_effects, axis=0)

print(
    f"mean of the country effect:\n{mean_country_effects}\n"
    f"std of the country effect:\n{std_country_effects}"
)

mean of the country effect:
[0.9193013  0.87520147 0.88159306 0.91016644 0.87867844 0.9256891
 0.89402616 0.84561655 0.95411406 0.85266475 0.87885591 0.8906819
 0.88087609 0.89921149 0.84880383 0.88943277 0.87963165 0.88938067
 0.89263848 0.87794319 0.87035789 0.92183316 0.8729789  0.85718749
 0.86514722 0.87801871 0.88988764 0.85262523 0.84206087 0.88758665
 0.88191222 0.9154826  0.90023098 0.88558454 0.92430319 0.90632543]
std of the country effect:
[0.40136349 0.35574208 0.38596043 0.39653429 0.44403531 0.47805652
 0.3084289  0.38626491 0.60597053 0.38010048 0.3636217  0.38168095
 0.33787248 0.44640337 0.35915354 0.34477914 0.37291452 0.3579008
 0.33734387 0.44447145 0.32918131 0.49730974 0.34615755 0.33905565
 0.32190713 0.36549689 0.32072042 0.41970452 0.35758934 0.37850256
 0.34007179 0.49458703 0.41931765 0.36275201 0.41966162 0.42023173]


In [389]:
mean_sector_effects = np.mean(sector_effects, axis=0)
std_sector_effects = np.std(sector_effects, axis=0)

print(
    f"mean of the sector effect:\n{mean_sector_effects}\n"
    f"std of the sector effect:\n{std_sector_effects}"
)

mean of the sector effect:
[0.95788112 0.92260925 0.93757961 0.97827551 0.95073891 1.0099772
 0.90939107 0.88169137 1.06447967 0.90109907 0.91412331 0.93425881
 0.93541858 0.96778969 0.89650196 0.90945918 0.94270648 0.92300363
 0.92741425 0.91241551 0.90514546 0.97711378 0.90378538 0.88887893
 0.90420213 0.89641506 0.87233076 0.88888865 0.87803274 0.91561035
 0.92359225 0.99299243 0.94250319 0.92582995 1.00195836 0.94517462]
std of the sector effect:
[0.429511   0.41310551 0.45103468 0.48637958 0.51789902 0.57122651
 0.30702743 0.39108825 0.74585856 0.41344715 0.3890334  0.43186139
 0.40879716 0.52552108 0.40485421 0.34605924 0.43412101 0.39657511
 0.37346518 0.42086689 0.35936563 0.51586838 0.37324673 0.364129
 0.35499138 0.3639509  0.30728834 0.41859174 0.37718893 0.39073478
 0.38139911 0.55430201 0.43041664 0.40229892 0.58376675 0.42569386]


In [390]:
mean_idiosyncratic_effects = np.mean(idiosyncratic_effects, axis=0)
std_idiosyncratic_effects = np.std(idiosyncratic_effects, axis=0)

print(
    f"mean of the idiosyncratic effect:\n{mean_idiosyncratic_effects}\n"
    f"std of the idiosyncratic effect:\n{std_idiosyncratic_effects}"
)

mean of the idiosyncratic effect:
[0.97507467 0.94637292 0.95634195 1.01090064 0.98418815 1.03451758
 0.90430683 0.89856145 1.11058587 0.91659796 0.92565065 0.95055528
 0.94168857 0.99308387 0.91066379 0.92211175 0.96001299 0.94188995
 0.93993304 0.93577996 0.91371911 1.00922768 0.91485256 0.90394496
 0.91448736 0.90773538 0.86413423 0.90347265 0.89062268 0.9387754
 0.93658188 1.01237083 0.96151991 0.94149802 1.02358101 0.96617463]
std of the idiosyncratic effect:
[0.42680615 0.43087538 0.46581399 0.52356874 0.56485303 0.57487046
 0.30390156 0.41959108 0.79382083 0.42756514 0.38241129 0.42061019
 0.38328093 0.54275916 0.41160757 0.35271858 0.45130165 0.39857505
 0.37590962 0.46271957 0.36100027 0.56283917 0.38181779 0.36088121
 0.35680741 0.3678142  0.29899482 0.4231673  0.38967015 0.40418987
 0.38659147 0.57374179 0.45301061 0.39397083 0.53633161 0.45138777]


In [393]:
country_list = data["country_list"]

In [397]:
import pandas as pd

# Create a DataFrame with one row per country and columns for each shock type summary
df = pd.DataFrame({
    "Country-Specific Mean": mean_country_effects,
    "Country-Specific Std": std_country_effects,
    "Sector-Specific Mean": mean_sector_effects,
    "Sector-Specific Std": std_sector_effects,
    "Idiosyncratic Mean": mean_idiosyncratic_effects,
    "Idiosyncratic Std": std_idiosyncratic_effects
}, index=country_list)

# Compute overall averages across countries for each summary statistic:
overall_means = df.mean(axis=0)
overall_stds = df.std(axis=0)

# Append overall average rows to the DataFrame.
# One row for the mean over all countries, and one for the standard deviation.
df.loc["Overall Mean"] = overall_means
df.loc["Overall Std"] = overall_stds

# Display the DataFrame
print(df)

# Output the DataFrame to an Excel file.
output_filename = "global_value_chain_effects.xlsx"
df.to_excel(output_filename)

              Country-Specific Mean  Country-Specific Std  \
AUS                        0.919301              0.401363   
AUT                        0.875201              0.355742   
BEL                        0.881593              0.385960   
BRA                        0.910166              0.396534   
BGR                        0.878678              0.444035   
CAN                        0.925689              0.478057   
CHN                        0.894026              0.308429   
CZE                        0.845617              0.386265   
DNK                        0.954114              0.605971   
EST                        0.852665              0.380100   
FIN                        0.878856              0.363622   
FRA                        0.890682              0.381681   
DEU                        0.880876              0.337872   
GRC                        0.899211              0.446403   
HUN                        0.848804              0.359154   
IND                     

## Summary of the Data Generating Process

We consider an international trade model with N countries and J sectors. In this simulation, we study three types of productivity shocks:
	•	Country-specific shocks:
Each country receives one independent productivity shock (drawn from a log-normal distribution with mean 0 and standard deviation 0.2), and this shock is assumed to affect all sectors in that country equally.
	•	Sector-specific shocks:
Each sector receives one independent productivity shock (drawn from the same log-normal distribution) that is common to all countries.
	•	Idiosyncratic shocks:
Every country-sector pair receives its own independent productivity shock.

For each shock type, we generate B=10 simulations. For each simulation, we compute the global value chain effect for each country as the relative change in the real wage when intermediate goods trade barriers increase (comparing the case dm=2 versus dm=1). Finally, for each shock type, we summarize the results by computing the mean and standard deviation (across the 10 simulations) of the global value chain effect for each country. These summary statistics are stored in arrays of shape (N,).

The results are then compiled into a Pandas DataFrame with one row per country (using a provided country name list) and columns for:
	•	Country-specific shock effect: mean and standard deviation,
	•	Sector-specific shock effect: mean and standard deviation,
	•	Idiosyncratic shock effect: mean and standard deviation,

as well as overall averages across countries for each measure.